# AtlanticWave SDX Cross-Domain Orchestration and Provisioning for Facilities and R&E Networks user interfaces with the SDXLib in Fabric Jupyter Notebook





# Import SDXLIB and FABLib Libraries

In [ ]:
from pprint import pprint
from sdxlib.sdx_token_auth import TokenAuthentication
from sdxlib.sdx_client import SDXClient
from sdxlib.sdx_exception import SDXException

# Instantiate the TokenAuthentication class with the correct token file path (if needed)

In [ ]:
# Read token from Jupyter Hub environment
import os

# token_file_path = "expired_id_token.json"
token_file_path = os.getenv("FABRIC_TOKEN_LOCATION", "/home/fabric/.tokens.json")

# Specify proxy_host
proxy_hostname = "sdxapi.atlanticwave-sdx.ai"

# Specify proxy_port
proxy_port = "443"

# Specify the endpoint
endpoint = "sax.net/sdx/topology"  # Replace with your desired endpoint

# Specify the name of the slice (provided during initialization)
slice_name = "Slice-AWSDX"

# Instantiate the TokenAuthentication class with the correct token file path and endpoint
token_auth = TokenAuthentication(
    token_path=token_file_path,
    proxy_hostname=proxy_hostname,
    proxy_port=proxy_port,
    endpoint=endpoint,
    slice_name=slice_name)

# Load the token from the file and Validate the token

In [ ]:
# Load the token from the file
token_auth.load_token()

# Print the decoded token for debugging (optional)
pprint(token_auth.token_decoded)

# Validate the token
pprint(token_auth.validate_token())

## Show the Slice's Attributes

In [ ]:
pprint(token_auth.show_slice())

## Configure the Subnet

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# Configure the interface(s) of the VM(s) with designated subnet
subnet = IPv4Network("192.168.1.0/24")
available_ips = list(subnet)[2:]

## Configure the Node

In [ ]:
node1 = slice.get_node(name=f"Node1")        
node1_iface = node1.get_interface(network_name=f'net_facility_port') 
node1_addr = available_ips.pop(99)
print(f"node1_addr: {node1_addr}")
node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet)

stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')

stdout, stderr = node1.execute(f'sudo ip link set dev {node1_iface.get_physical_os_interface_name()} up')

stdout, stderr = node1.execute(f'sudo ip link set dev {node1_iface.get_os_interface()} up')

## List Nodes

In [ ]:
slice.list_nodes()


## List Networks

In [ ]:
slice.list_networks()


## List Interfaces

In [ ]:
slice.list_interfaces()

## Ping a node on the Amlight Facility Port side

In [ ]:
stdout, stderr = node1.execute(f'ping -c 5 192.168.1.10')   

## Assign SDX Controller URL

In [ ]:
url = "http://controller.atlanticwave-sdx.net:8080/SDX-Controller"

## List SDX Domain Nodes and Ports


In [ ]:
client=SDXClient(url)
client.get_available_ports()

## Create SDX Layer 2 VPN

The below specification is the minimum information needed to create the layer 2 VPN. 

In [ ]:
client_name = "Test SDX SC24 L2VPN"
client_endpoints = [
    {"port_id": "urn:sdx:port:amlight.net:MIA-MI1-SW17:7", "vlan": "4015"}, 
    {"port_id": "urn:sdx:port:amlight.net:MIA-MI1-SW15:9", "vlan": "4015"},
]
client = SDXClient(url, client_name, client_endpoints)

In [ ]:
try:
    response = client.create_l2vpn()  
    print("L2VPN creation successful!")
    pprint(response)
except SDXException as e:
    print(f"L2VPN creation failed: {e}") 

## List All SDX L2VPNs

Both of the list methods have the option of query results being listed as a DataFrame(default) or as a JSON by passing the optional argument format='json'.

In [ ]:
client.get_all_l2vpns()

## Show created L2VPN Details

In [ ]:
client.get_l2vpn(service_id = response['service_id'])

## Ping a node on the Amlight Facility Port side

In [ ]:
stdout, stderr = node1.execute(f'ping -c 5 192.168.1.10')   

## Delete created SDX L2VPN

Please make certain to delete when you are finished. 

In [ ]:
client.delete_l2vpn(service_id = response['service_id'])

## Ping a node on the Amlight Facility Port side

In [ ]:
stdout, stderr = node1.execute(f'ping -c 5 192.168.1.10')   